Assignment 7: Neural Networks using Keras and Tensorflow Please see the associated document for questions

If you have problems with Keras and Tensorflow on your local installation please make sure they are updated. On Google Colab this notebook runs.

In [67]:
# imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Lambda, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
from matplotlib import pyplot as plt

In [68]:
# Hyper-parameters data-loading and formatting

batch_size = 128
num_classes = 10
epochs = 10

img_rows, img_cols = 28, 28

(x_train, lbl_train), (x_test, lbl_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

**Preprocessing**

In [69]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = tf.keras.utils.to_categorical(lbl_train, num_classes)
y_test = tf.keras.utils.to_categorical(lbl_test, num_classes)




In [ ]:
## Define model ##
model = Sequential()

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(num_classes, activation='softmax'))


model.compile(loss=keras.losses.categorical_crossentropy,
               optimizer=tf.keras.optimizers.SGD(lr = 0.1),
        metrics=['accuracy'],)

fit_info = model.fit(x_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {}, Test accuracy {}'.format(score[0], score[1]))

In [ ]:
#plotting
ep = [1,2,3,4,5,6,7,8,9,10]
train_accuracy = fit_info.history['accuracy']
val_accuracy = fit_info.history['val_accuracy']

plt.plot(ep,train_accuracy, label = "train accuracy")
plt.plot(ep,val_accuracy, label = "val accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

### Question 4) Auto-Encoder for denoising


In [71]:
import numpy as np
def salt_and_pepper(input, noise_level=0.5):
    """
    This applies salt and pepper noise to the input tensor - randomly setting bits to 1 or 0.
    Parameters
    ----------
    input : tensor
        The tensor to apply salt and pepper noise to.
    noise_level : float
        The amount of salt and pepper noise to add.
    Returns
    -------
    tensor
        Tensor with salt and pepper noise applied.
    """
    # salt and pepper noise
    a = np.random.binomial(size=input.shape, n=1, p=(1 - noise_level))
    b = np.random.binomial(size=input.shape, n=1, p=0.5)
    c = (a==0) * b
    return input * a + c


#data preparation
flattened_x_train = x_train.reshape(-1,784)
flattened_x_train_seasoned = salt_and_pepper(flattened_x_train, noise_level=0.4)

flattened_x_test = x_test.reshape(-1,784)
flattened_x_test_seasoneed = salt_and_pepper(flattened_x_test, noise_level=0.4)


In [72]:
def increase_noise(test,noise):
  x_test = salt_and_pepper(test,noise_level = noise)

  return x_test


In [78]:
latent_dim = 96  

input_image = keras.Input(shape=(784,))
encoded = Dense(128, activation='relu')(input_image)
encoded = Dense(latent_dim, activation='relu')(encoded)


decoded = Dense(128, activation='relu')(encoded) 
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = keras.Model(input_image, decoded) 
encoder_only = keras.Model(input_image,encoded) 

encoded_input = keras.Input(shape=(latent_dim,))
decoder_layer = Sequential(autoencoder.layers[-2:])
decoder = keras.Model(encoded_input, decoder_layer(encoded_input))


autoencoder.compile(optimizer='adam', loss = 'binary_crossentropy')

In [ ]:
## changed input
fit_info_AE = autoencoder.fit(flattened_x_train_seasoned, flattened_x_train,
                epochs=32,
                batch_size=64,
                shuffle=True,
                validation_data=(flattened_x_test_seasoneed, flattened_x_test))


### Code for generating random images: original, seasoned and denoised

In [ ]:

x_seasoned = increase_noise(flattened_x_test,noise = 0.8)


num_images = 4
np.random.seed(42)
random_test_images = np.random.randint(flattened_x_test.shape[0], size=num_images)

decoded_img = autoencoder.predict(x_seasoned)

plt.figure(figsize=(4, 4))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(3, num_images, i + 1)
    plt.imshow(flattened_x_test[image_idx].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # plot seasoned image
    ax = plt.subplot(3, num_images, num_images + i + 1)
    plt.imshow(x_seasoned[image_idx].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # plot reconstructed image
    ax = plt.subplot(3, num_images, 2*num_images + i + 1)
    plt.imshow(decoded_img[image_idx].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

plt.show()

### Classification using denoising

In [11]:
import pickle
loaded_model = pickle.load(open('final_model.sav', 'rb'))


In [ ]:

noise = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

score_noise = []
score_denoise = []

for i in noise:

  # adding noise
  train_noise = salt_and_pepper(x_train,i)
  test_noise = salt_and_pepper(x_test,i)

  #denoising
  train_denoised = autoencoder.predict(train_noise.reshape(-1,784))
  test_denoised = autoencoder.predict(test_noise.reshape(-1,784))

  train_denoised = train_denoised.reshape(60000,28,28,1)
  test_denoised = test_denoised.reshape(10000,28,28,1)

  # fit model with noisy data
  fit_noise = model.fit(train_noise, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(test_noise, y_test))
  score_noise.append(model.evaluate(test_noise, y_test, verbose=0)[1])

  # fit model with denoised data
  fit_denoise = model.fit(train_denoised, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(test_denoised, y_test))
  score_denoise.append(model.evaluate(test_denoised, y_test, verbose=0)[1])






In [ ]:
# plotting noise vs denoised
plt.plot(noise,score_noise, label = "noise")
plt.plot(noise,score_denoise, label = "denoise")
plt.xlabel('Noise-level')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
